In [63]:
from pymongo import MongoClient
import json

client = MongoClient('localhost',27017)

db = client['stationdata']

stations=db['stations']

stations.ensure_index([("loc","2dsphere")])

u'loc_2dsphere'

In [64]:
from GTOPO30 import getElevationProfile

help(getElevationProfile)

Help on function getElevationProfile in module GTOPO30:

getElevationProfile(x0, y0, x1, y1)



In [110]:
import re
import numpy as np
from pandas import DataFrame
from GTOPO30 import getElevationProfile
from LatLon import LatLon

for station in stations.find({
        "_id":re.compile('^KCASANFR')
    }).limit(5):
    # Get neearby stations
    neighborQuery = stations.find({
            "_id":{"$ne":station['_id']},
            "loc":{"$near":{"$geometry":station['loc'],
               "$maxDistance":3000}
              }
        }).limit(5)
    if neighborQuery.count()==np.nan:
        print "Station %s has no neighbors."%station['_id']
    else:
        print "%s's neighbors:"%station['_id']
        c0 = station['loc']['coordinates']
        stationLoc = LatLon(c0[1],c0[0])
        for neighbor in neighborQuery:
            c1 = neighbor['loc']['coordinates']
            elevProfile = DataFrame(getElevationProfile(c0[0],c0[1],c1[0],c1[1]),
                                    columns=('lon','lat','elev'))
            elevs=elevProfile['elev']
            elevs[elevs==-9999]=0  # set no-data to 0
            #print (elevProfile)
            relativePeakHeight = elevs.max() \
                - np.max([elevs[0],elevs[len(elevProfile)-1]])
            neighborLoc = LatLon(c1[1],c1[0])
            dist = stationLoc.distance(neighborLoc)
            # TODO:
            #  - check relativePeakHeight against threshold
            #  - check proximity to water
            
            print(neighbor['_id'], elevs.max(),relativePeakHeight,dist)

KCASANFR100's neighbors:
(u'KCASANFR409', 169, 0, 1.2665936541670024)
(u'KCADALYC10', 60, 0, 1.9700515459539902)
(u'KCASANFR69', 54, 0, 2.09638632030956)
(u'KCASANFR286', 54, 0, 2.1107234668178485)
(u'KCASANFR503', 143, 0, 2.343634249449251)
KCASANFR102's neighbors:
(u'KCASANFR236', 58, 0, 0.4513471975187209)
(u'KCASANFR564', 28, 0, 0.4915132298281701)
(u'MPXOC1', 17, 0, 0.7427069960804095)
(u'BUOYPXOC1', 17, 0, 0.7427069960804095)
(u'KCASANFR549', 41, 0, 0.8257420796958704)
KCASANFR107's neighbors:
(u'KCASANFR99', 60, 0, 0.3628477282815465)
(u'KCASANFR545', 21, 3, 1.4740860263336595)
(u'KCASANFR302', 49, 0, 1.7360440108889432)
(u'KCASANFR577', 9, 9, 1.9424675149028297)
(u'KCASANFR97', 117, 62, 2.401585877587769)
KCASANFR110's neighbors:
(u'KCASANFR374', 121, 0, 0.32921132350909443)
(u'KCASANFR382', 121, 0, 0.347037092168184)
(u'KCASANFR392', 121, 0, 0.3914080970842387)
(u'KCASANFR498', 121, 0, 0.5003745839531057)
(u'KCASANFR388', 121, 0, 0.665680214683147)
KCASANFR111's neighbors:
(u'

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
